# Illinois Truck Sales 2018-2020: Mileage vs. Price Regressions

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

### Import CSV and Examine Data

In [2]:
# Import Data Into Pandas DataFrame

file = 'SMU_DATA\csv\Pickup_Data_Tot.csv'
df_all = pd.read_csv(file)

In [3]:
df_all.dtypes

vf_BodyClass             object
brandName                object
vf_Make                  object
modelName                object
vf_Model                 object
vf_ModelID              float64
vf_ModelYear            float64
mileage                   int64
askPrice                  int64
msrp                      int64
vf_BasePrice            float64
askPrice.1                int64
color                    object
interiorColor            object
vf_BodyCabType           object
vf_Doors                float64
vf_EngineCylinders      float64
vf_EngineHP             float64
vf_FuelTypePrimary       object
vf_FuelTypeSecondary     object
vf_Series                object
vf_Series2               object
vf_SteeringLocation      object
vf_TransmissionStyle     object
vf_Trim                  object
vf_Trim2                 object
vf_Turbo                 object
vin                      object
dtype: object

In [4]:
df_all.describe()

,vf_ModelID,vf_ModelYear,mileage,askPrice,msrp,vf_BasePrice,askPrice.1,vf_Doors,vf_EngineCylinders,vf_EngineHP
count,216790.000000,216808.000000,2.168080e+05,2.168080e+05,2.168080e+05,24505.000000,2.168080e+05,68762.000000,210117.000000,99790.000000
mean,3825.303741,2012.795686,7.466362e+04,4.374941e+05,4.586837e+05,37789.820567,4.374941e+05,3.626742,7.292437,336.290075
std,4428.621520,5.151297,6.625956e+04,2.988625e+07,3.058941e+07,6808.996063,2.988625e+07,0.779233,1.070932,69.092071
min,1801.000000,1986.000000,1.000000e+03,0.000000e+00,0.000000e+00,18990.000000,0.000000e+00,2.000000,4.000000,80.000000
25%,1801.000000,2010.000000,2.774800e+04,1.049500e+04,1.099900e+04,33650.000000,1.049500e+04,4.000000,6.000000,300.000000
50%,1850.000000,2015.000000,5.754300e+04,2.383900e+04,2.499000e+04,35790.000000,2.383900e+04,4.000000,8.000000,365.000000
75%,1941.000000,2017.000000,1.093428e+05,3.097200e+04,3.289000e+04,42870.000000,3.097200e+04,4.000000,8.000000,386.000000
max,25919.000000,2021.000000,9.999999e+06,2.147484e+09,2.147484e+09,60495.000000,2.147484e+09,4.000000,10.000000,450.000000


In [5]:
print('Number of Records: '+str(len(df_all['vin'])))

Number of Records: 216808


### Review Prices Above $0

In [6]:
df_clean = df_all.loc[df_all['askPrice']>0]
df_clean.head(51)

,vf_BodyClass,brandName,vf_Make,modelName,vf_Model,vf_ModelID,vf_ModelYear,mileage,askPrice,msrp,...,vf_FuelTypePrimary,vf_FuelTypeSecondary,vf_Series,vf_Series2,vf_SteeringLocation,vf_TransmissionStyle,vf_Trim,vf_Trim2,vf_Turbo,vin
4,Pickup,FORD,FORD,F-250,F-250,1805.0,2017.0,19106,40962,40962,...,Diesel,NaN,Super Duty - Single Rear Wheel,NaN,Left Hand Drive (LHD),NaN,NaN,NaN,NaN,bcd4bbecbba3b9ed2330d543abc3d90b997aa8fa6e628d...
7,Pickup,FORD,FORD,F-150,F-150,1801.0,2017.0,28194,28962,28962,...,Gasoline,NaN,NaN,NaN,Left Hand Drive (LHD),NaN,SuperCrew,NaN,NaN,a8f09496d9523e33ee3a8f885a475f5ef001d68a334c98...
9,Pickup,GMC,GMC,Sierra,Sierra,1857.0,2017.0,7483,43962,43962,...,Gasoline,NaN,NaN,NaN,Left Hand Drive (LHD),NaN,1500 SLT,NaN,NaN,c65dd3e96f4f68a0d4c7b09005273e0312295057c7c529...
10,Pickup,FORD,FORD,F-250,F-250,1805.0,2017.0,23881,38962,38962,...,Diesel,NaN,Super Duty - Single Rear Wheel,NaN,Left Hand Drive (LHD),NaN,NaN,NaN,NaN,966168515802a7cf510d6c87a15cf71cc4385316a504c6...
18,Pickup,GMC,GMC,Canyon,Canyon,4093.0,2017.0,6333,30962,30962,...,Gasoline,NaN,SLE,NaN,Left Hand Drive (LHD),NaN,NaN,NaN,NaN,9c7648c843035d0b46e9d716d947dfa3d0e8881054247a...
21,Pickup,FORD,FORD,F-150,F-150,1801.0,2016.0,26519,33962,33962,...,Gasoline,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a7744ea6d085d017db4eb549ed430a4d4b8074c9cb1112...
32,Pickup,FORD,FORD,F-150,F-150,1801.0,2017.0,21638,30962,30962,...,Gasoline,NaN,NaN,NaN,Left Hand Drive (LHD),NaN,SuperCrew,NaN,NaN,258da8b91266c89798249877ed79bec864e49648eabd80...
43,Pickup,FORD,FORD,F-150,F-150,1801.0,2018.0,8213,56962,56962,...,Gasoline,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fca4164c6db70e3e79ada174ef23a362cac1b1429f869b...
45,Pickup,FORD,FORD,F-150,F-150,1801.0,2016.0,23695,41962,41962,...,Gasoline,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cd4ba8b2c47e66a96b02a95521c480f918e7bd3a22944e...
51,Pickup,FORD,FORD,F-150,F-150,1801.0,2016.0,32084,28962,28962,...,Gasoline,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34ad4bc5a8e0496ae7ed3315c2abda14eb2901f40615b1...


In [7]:
print('Number of Records: '+str(len(df_clean['vin'])))

Number of Records: 183226


### Convert 'nan' to 'NP.NaN' (Numpy Missing Value Format)

In [8]:
df_clean1 = df_clean.replace('nan',np.nan)
df_clean1.head(50)

,vf_BodyClass,brandName,vf_Make,modelName,vf_Model,vf_ModelID,vf_ModelYear,mileage,askPrice,msrp,...,vf_FuelTypePrimary,vf_FuelTypeSecondary,vf_Series,vf_Series2,vf_SteeringLocation,vf_TransmissionStyle,vf_Trim,vf_Trim2,vf_Turbo,vin
4,Pickup,FORD,FORD,F-250,F-250,1805.0,2017.0,19106,40962,40962,...,Diesel,NaN,Super Duty - Single Rear Wheel,NaN,Left Hand Drive (LHD),NaN,NaN,NaN,NaN,bcd4bbecbba3b9ed2330d543abc3d90b997aa8fa6e628d...
7,Pickup,FORD,FORD,F-150,F-150,1801.0,2017.0,28194,28962,28962,...,Gasoline,NaN,NaN,NaN,Left Hand Drive (LHD),NaN,SuperCrew,NaN,NaN,a8f09496d9523e33ee3a8f885a475f5ef001d68a334c98...
9,Pickup,GMC,GMC,Sierra,Sierra,1857.0,2017.0,7483,43962,43962,...,Gasoline,NaN,NaN,NaN,Left Hand Drive (LHD),NaN,1500 SLT,NaN,NaN,c65dd3e96f4f68a0d4c7b09005273e0312295057c7c529...
10,Pickup,FORD,FORD,F-250,F-250,1805.0,2017.0,23881,38962,38962,...,Diesel,NaN,Super Duty - Single Rear Wheel,NaN,Left Hand Drive (LHD),NaN,NaN,NaN,NaN,966168515802a7cf510d6c87a15cf71cc4385316a504c6...
18,Pickup,GMC,GMC,Canyon,Canyon,4093.0,2017.0,6333,30962,30962,...,Gasoline,NaN,SLE,NaN,Left Hand Drive (LHD),NaN,NaN,NaN,NaN,9c7648c843035d0b46e9d716d947dfa3d0e8881054247a...
21,Pickup,FORD,FORD,F-150,F-150,1801.0,2016.0,26519,33962,33962,...,Gasoline,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a7744ea6d085d017db4eb549ed430a4d4b8074c9cb1112...
32,Pickup,FORD,FORD,F-150,F-150,1801.0,2017.0,21638,30962,30962,...,Gasoline,NaN,NaN,NaN,Left Hand Drive (LHD),NaN,SuperCrew,NaN,NaN,258da8b91266c89798249877ed79bec864e49648eabd80...
43,Pickup,FORD,FORD,F-150,F-150,1801.0,2018.0,8213,56962,56962,...,Gasoline,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fca4164c6db70e3e79ada174ef23a362cac1b1429f869b...
45,Pickup,FORD,FORD,F-150,F-150,1801.0,2016.0,23695,41962,41962,...,Gasoline,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cd4ba8b2c47e66a96b02a95521c480f918e7bd3a22944e...
51,Pickup,FORD,FORD,F-150,F-150,1801.0,2016.0,32084,28962,28962,...,Gasoline,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34ad4bc5a8e0496ae7ed3315c2abda14eb2901f40615b1...


In [9]:
df_clean1['color'].isnull().sum()

4917

### Missing Data and Unique Value Analysis

In [10]:
col_list = df_clean1.columns.tolist()
num_col = len(col_list)
num_nan_list = []
num_unq_list = []
per_nan_list = []

for x in range(num_col):
    num_nan_list.append(df_clean1[col_list[x]].isnull().sum())

for x in range(num_col):
    num_unq_list.append(df_clean1[col_list[x]].nunique())
    
for x in range(num_col):
    per_nan_list.append('{:,.2f}%'.format(((num_nan_list[x])/(len(df_clean1['vin'])))*100))

analysis_dict = {'Column Name':col_list,
                 'Number Missing':num_nan_list,
                 'Percentage Missing': per_nan_list,
                 'Number Unique Items': num_unq_list
                 }
analysis_df = pd.DataFrame(analysis_dict)
analysis_df.set_index(['Column Name'], inplace=True)
analysis_df.head(31)

,Number Missing,Percentage Missing,Number Unique Items
Column Name,,,
vf_BodyClass,0,0.00%,1
brandName,0,0.00%,16
vf_Make,0,0.00%,16
modelName,15,0.01%,45
vf_Model,15,0.01%,45
vf_ModelID,15,0.01%,45
vf_ModelYear,0,0.00%,35
mileage,0,0.00%,50776
askPrice,0,0.00%,17802


In [11]:
# Analyze Unique Values in BodyCabType
# Remove As Identifiers are Not Descrptive Enough

df_clean1.groupby('vf_BodyCabType')['vin'].count()

vf_BodyCabType
Crew/ Super Crew/ Crew Max                104968
Extra/Super/ Quad/Double/King/Extended     54388
Mega                                         153
Regular                                     6035
Name: vin, dtype: int64

### Keep Only Rows with Less than 5% Missing Data

In [12]:
df_clean2 = df_clean1[['brandName',
                       'modelName',
                       'vf_ModelID',
                       'vf_ModelYear',
                       'mileage',
                       'askPrice',
                       'msrp',
                       'color',
                       'vf_EngineCylinders',
                       'vf_FuelTypePrimary',
                       'vin'
                      ]]
df_clean2.head(50)

,brandName,modelName,vf_ModelID,vf_ModelYear,mileage,askPrice,msrp,color,vf_EngineCylinders,vf_FuelTypePrimary,vin
4,FORD,F-250,1805.0,2017.0,19106,40962,40962,Ingot Silver Metallic,8.0,Diesel,bcd4bbecbba3b9ed2330d543abc3d90b997aa8fa6e628d...
7,FORD,F-150,1801.0,2017.0,28194,28962,28962,Ingot Silver Metallic,6.0,Gasoline,a8f09496d9523e33ee3a8f885a475f5ef001d68a334c98...
9,GMC,Sierra,1857.0,2017.0,7483,43962,43962,Crimson Red Tintcoat,8.0,Gasoline,c65dd3e96f4f68a0d4c7b09005273e0312295057c7c529...
10,FORD,F-250,1805.0,2017.0,23881,38962,38962,Oxford White,8.0,Diesel,966168515802a7cf510d6c87a15cf71cc4385316a504c6...
18,GMC,Canyon,4093.0,2017.0,6333,30962,30962,Onyx Black,6.0,Gasoline,9c7648c843035d0b46e9d716d947dfa3d0e8881054247a...
21,FORD,F-150,1801.0,2016.0,26519,33962,33962,Oxford White,8.0,Gasoline,a7744ea6d085d017db4eb549ed430a4d4b8074c9cb1112...
32,FORD,F-150,1801.0,2017.0,21638,30962,30962,Shadow Black,8.0,Gasoline,258da8b91266c89798249877ed79bec864e49648eabd80...
43,FORD,F-150,1801.0,2018.0,8213,56962,56962,White Platinum Metallic Tri-Coat,6.0,Gasoline,fca4164c6db70e3e79ada174ef23a362cac1b1429f869b...
45,FORD,F-150,1801.0,2016.0,23695,41962,41962,Blue Flame Metallic,6.0,Gasoline,cd4ba8b2c47e66a96b02a95521c480f918e7bd3a22944e...
51,FORD,F-150,1801.0,2016.0,32084,28962,28962,Oxford White,8.0,Gasoline,34ad4bc5a8e0496ae7ed3315c2abda14eb2901f40615b1...


### Drop Rows If Any Columns Are Missing Data

In [13]:
df_clean2.dropna(axis=0, how='any', inplace=True)

<ipython-input-13-4c73de67bed6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean2.dropna(axis=0, how='any', inplace=True)


In [14]:
print('Number of Records: '+str(len(df_clean2['vin'])))

Number of Records: 169190


### Double Check For Missing Values

In [15]:
col_list = df_clean2.columns.tolist()
num_col = len(col_list)
num_nan_list = []
num_unq_list = []
per_nan_list = []

for x in range(num_col):
    num_nan_list.append(df_clean2[col_list[x]].isnull().sum())

for x in range(num_col):
    num_unq_list.append(df_clean2[col_list[x]].nunique())
    
for x in range(num_col):
    per_nan_list.append('{:,.2f}%'.format(((num_nan_list[x])/(len(df_clean2['vin'])))*100))

analysis_dict = {'Column Name':col_list,
                 'Number Missing':num_nan_list,
                 'Percentage Missing': per_nan_list,
                 'Number Unique Items': num_unq_list
                 }
analysis_df = pd.DataFrame(analysis_dict)
analysis_df.set_index(['Column Name'], inplace=True)
analysis_df.head(31)

,Number Missing,Percentage Missing,Number Unique Items
Column Name,,,
brandName,0,0.00%,15
modelName,0,0.00%,44
vf_ModelID,0,0.00%,44
vf_ModelYear,0,0.00%,35
mileage,0,0.00%,47696
askPrice,0,0.00%,17020
msrp,0,0.00%,16278
color,0,0.00%,1526
vf_EngineCylinders,0,0.00%,5


### Drop Duplicate VIN Numbers

In [16]:
df_clean3 = df_clean2.drop_duplicates(subset='vin',keep='first')
print('Number of Records: '+str(len(df_clean3['vin'])))

Number of Records: 56017


In [17]:
df_clean3.to_csv('SMU_DATA/csv/Pickup_Data_Clean.csv',index=False,header=True)
print('File_Saved')

File_Saved


In [18]:
df_clean3.describe()

,vf_ModelID,vf_ModelYear,mileage,askPrice,msrp,vf_EngineCylinders
count,56017.00000,56017.000000,5.601700e+04,5.601700e+04,5.601700e+04,56017.000000
mean,4038.80879,2013.644215,6.584410e+04,1.099595e+06,1.101296e+06,7.263402
std,4600.47874,4.406154,5.222448e+04,4.799975e+07,4.799973e+07,1.062178
min,1801.00000,1986.000000,1.000000e+03,1.000000e+00,1.000000e+00,4.000000
25%,1801.00000,2012.000000,2.629700e+04,1.850000e+04,1.950000e+04,6.000000
50%,1850.00000,2015.000000,5.055500e+04,2.689600e+04,2.799800e+04,8.000000
75%,2223.00000,2017.000000,9.366100e+04,3.286100e+04,3.422500e+04,8.000000
max,25919.00000,2020.000000,1.955308e+06,2.147484e+09,2.147484e+09,10.000000
